# POLSCI 3

## Week 11, Lecture Notebook 1: Multivariate Regression with Observational Data

### What is multivariate regression?

Last week, I mentioned that regression is _by far_ the most common tool social scientists use to analyze data.

So far, we've seen _bivarate regression_, which uses just one predictor to predict the outcome, like this:

`summary(lm(outcome ~ predictor, data = data))`

In this case, regression finds the values of $m$ and $b$ in $y = mx + b$ that give us the best predictions about what y will be given a value of x. This was really easy to visualize as a scatter plot with a line of best fit.

But we can actually use _multiple variables_ to predict an outcome, with code like this:

`summary(lm(outcome ~ predictor.1 + predictor.2, data = data))`

Instead of $y = mx + b$, we instead have $y = m_1x_1 + m_2x_2 + b$, where we are adding more variables. In this case, regression does the same thing as it did before: it asks what values of $m_1$, $m_2$, and $b$ mean we'll make the best predictions about $y$ given values of $x_1$ and $x_2$.

For example, say we want to predict what percentage of people in Berkeley are wearing coats on a given day. This is probably influenced by both temperature and whether it's raining. Suppose we had a dataset on whether it was raining and what the temperature was every day in Berkeley. We can run separate regressions like:

`summary(lm(how.many.people.wearing.coats ~ temperature, data = data))`
`summary(lm(how.many.people.wearing.coats ~ inches.of.rain, data = data))`

But we can also just run one big regression, like this:

`summary(lm(how.many.people.wearing.coats ~ temperature + inches.of.rain, data = data))`

This would give us a formula for how to best predict the number of people wearing coats given a temperature and how many inches of rain fell that day.

If you work at the Berkeley Student Store and are trying to predict how many coats to stock on your shelves based on a weather forecast, this kind of prediction could be really useful. Again, remember, this is all just about making predictions, not causality.

So, this is the first use of multivariate regression: making better predictions about a $y$ variable using multiple $x$ varaibles.

### "Controlling for" other factors

But there is a second use: when trying to infer causality, reducing _omitted variable bias_. In particular, multivariate regression is also social scientists' most common tool for trying to reduce _omitted variable bias_ when making claims about causality.

The $m_1$, $m_2$, and so on (you can have as many variables on the right hand side as you want) estimates we get tell us the predictions we'd make about how $y$ would change as $x_1$, $x_2$, or the other $x$ variables change _when holding constant all the other $x$ variables_.

This can be helpful for reducing omitted variable bias, since we can examine the relationship between _one of_ our $x$ variables and $y$ while _holding constant_ other $x$ variables we include in the regression. This means holding constant potential omitted variables.

For example, suppose someone ran this regression to argue that rain _causes_ people to wear coats:

`summary(lm(how.many.people.wearing.coats ~ inches.of.rain, data = data))`

A potential omitted variable is temperature: it's usually cold when it's raining, and it could be that when we compare rainy days to non-rainy days, what we're really comparing is warmer days (when it's less likely to rain) and colder days (when it's more likely to rain). By including `temperature` in the regression, like this:

`summary(lm(how.many.people.wearing.coats ~ temperature + inches.of.rain, data = data))`

we can now get estimates of the relationship between rain and whether people are wearing coats _while holding temperature constant_, no longer having to worry that it is an omitted variable.

#### Just because you've controlled for one omitted variable doesn't mean you've gotten rid of them all

Multivariate regression is the most _dangerous_ tool I'll teach you in this class.

<img src="danger.png" />

It's dangerous because _just because you've controlled for one omitted variable doesn't mean you've gotten rid of them all_, but people often mistakely think this. Multivariate regression does _not_ solve the causal inference problem the way that experiments do, but social scientists often mistakenly assume that it does.

The tricky thing about omitted variable bias is that we never know if we've thought of all the possible omitted variables. Think back to some of the open-ended questions in Week 3 in this class: every group was able to think of a different omitted variable that could confound the correlations we saw between variables like a country's income and its life expectancy. So, **don't assume that just because you've controlled for some omitted variables that you've gotten rid of all omitted variable bias -- there could be others!**

With that important warning in mind, let's get a feel for how multivariate regression works in practice:

### Data

This week's data is data on US House elections in 2018 and 2020.

In this data, every row is one Congressional district. There are 435 Congressional districts in the US, and each one has one representative in the US House of Representatives. For example, UC Berkeley is in California District 13, which looks like this:

<img src="cd13.tiff" />

and Barbara Lee is our representative. So, one row in this dataset is for CA-13 and Barbara Lee.

Let's read in the data:

In [ ]:
library(ggplot2) # Allow us to make some graphs later

data <- read.csv("ps3-house-election-spending.csv")
head(data)

Here is a quick rundown of what each column means:

- `state`: State (e.g., for CA-13, "CA")
- `district`: District number (e.g., for CA-13, 13)
- `name_dem_cand`: Democrat candidate name in the 2020 US House elections
- `name_rep_cand`: Republican candidate name in the 2020 US House elections
- `dem_us_house_percent_2020`: Democrat candidate's vote share in 2020 election (percent)
- `dem_us_house_percent_2018`: Democrat candidate's vote share in 2018 election (percent)
- `dem_won_ushouse_2018`: A Democrat won the US House election in 2018, and so is running for re-election in 2020 (0 = lost, 1 = won)
- `clinton_percent_2016`: Clinton vote share in 2016 in the district (percent)
- `spending_dem_ushouse_2020`: Democratic US House candidate's spending in 2020, in millions of dollars 
- `spending_rep_ushouse_2020`: Republican US House candidate's spending in 2020, in millions of dollars 

### Does campaign spending work?

We'll use this data to see how multivariate regression can help us _try_ (not necessarily succeed) at reducing omitted variable bias when trying to understand the effects of campaign spending in US elections.

To make things easy, let's begin by subsetting the data to Democratic districts (where Democrats won the house election in 2018):

In [ ]:
democrats <- subset(data, dem_won_ushouse_2018 == 1)

Using bivariate regression we learned last week, what seems to be the "effect" of spending among Democratic incumbents? **Note: this regression is susceptible to omitted variable bias!!**

In [ ]:
summary(lm(dem_us_house_percent_2020 ~ spending_dem_ushouse_2020, democrats))

Weird! For every \$1 million a Democratic Member of the US House spends on their re-election campaign, we predict that they do 1.2 percentage points *worse* in the upcoming election!

This is probably a result of omitted variable bias. Let's think, what might cause the "treatment" here (Democrats to spend money on their campaigns for re-election) and the outcome (Democrats' performance in the election).

One possibility: underlying partisan composition of the district. Some districts are evenly divided between Democratic and Republican voters. In these districts, Democrats only get around half the votes, and Democratic politicians spend a lot of money on campaigning since the election is so close. On the other hand, in "safe" districts (e.g., our own CA-13; Barbara Lee won with 90.4\% of the vote in 2020), Democrats do well and they don't need to spend much money on their campaign.

In other words: incumbents who are in more competitive districts raise and spend more since they're more worried about losing. Incumbents in "safe seats" spend less. This makes incumbents who do less well in the election spend more, not because spending causes them to do worse, but because anticipating they will do worse causes them to spend!

#### Can multivariate regression fix this problem? (Spoiler alert: only partly)

Let's apply the idea of multivariate regression: hold constant these omitted variables, such as how "close" their district is. For example, let's control for how Democratic-leaning the district was in the 2016 Presidential election, _and_ how they performed in the last election.

In [ ]:
summary(lm(dem_us_house_percent_2020 ~ spending_dem_ushouse_2020 +
           clinton_percent_2016 + dem_us_house_percent_2018, democrats))

Now, the effect of spending is statistically insignificant and the estimate is much smaller. But, we probably haven't solved omitted variable bias.

So, the **most important thing to remember**: you can never be sure you've solved omitted variable bias with multivariate regression. There could still be other possible omitted variables you're not thinking about. For example, another omitted variable could be the _quality of the challenger_: in some districts, a well-known or well-liked Republican might decide to run in 2020, and this might cause Democrats to spend more and to do less well in the election. How well-known or well-liked a Republican challenger is is quite difficult to measure, though; it's unlikely we would ever do a good job controlling for that potential omitted variable.

### How exactly does multivariate regression "control" for other factors?

How multivariate regression works is actually really complicated. The fact that multivariate regression is so complicated is another red flag that you shouldn't assume it totally solves the omitted variable bias problem: if it's hard to understand what it is doing, how can we be sure it's doing the right thing?

But I do want to walk through a quick example to give you a bit of intuition about what is happening. **You don't need to learn the R commands in the rest of this notebook, but I do want you to understand the basic idea.**

Let's first start with a regression predicting how Democrat did in the 2020 election based on how Clinton did in the 2016 election in that same district, which is a way to measure the partisan lean of that district:

In [ ]:
spending.lm <- lm(dem_us_house_percent_2020 ~ clinton_percent_2016, democrats)
summary(spending.lm)

Here's what is happening under the hood of that regression:

In [ ]:
# It took this relationship:
qplot(clinton_percent_2016, dem_us_house_percent_2020, data = democrats)

In [ ]:
# ...the regression forms a "best guess" for every point, a prediction:
democrats$predicted.dem_us_house_percent_2020 <- predict(spending.lm)

# And found the best prediction based on drawing a line:
qplot(clinton_percent_2016, predicted.dem_us_house_percent_2020, data = democrats)

Let's see both on the same graph.

In [ ]:
ggplot(democrats, aes(x = clinton_percent_2016, y = dem_us_house_percent_2020)) +
geom_segment(aes(xend = clinton_percent_2016, yend = predicted.dem_us_house_percent_2020)) +
  geom_point() +
  geom_point(aes(y = predicted.dem_us_house_percent_2020), shape = 1)

The lines I've drawn are called residuals. It's the information left over in the $y$ variable that can't be explained based on the first $x$ variable. That is, a multivariate regression "takes out the influence" of the other predictors, leaving only the residuals left. That's why they're called residuals - it's what is left over.

In [ ]:
# The lines on the graph are called residuals.
democrats$residuals <- resid(spending.lm)

Let's next look at the relationship between spending and performance in 2020 *without* controlling for the underlying partisan lean of the district:

In [ ]:
qplot(spending_dem_ushouse_2020, dem_us_house_percent_2020, data = democrats) + 
    geom_smooth(method = 'lm') 

But what if, instead of looking at the correlation between spending and how Democrats did in 2020 (as above), we *remove* the part of `dem_us_house_percent_2020` that can be explained by the partisan lean of the district? That is what the residuals contain.

And we can see that, when we removed the part of `spending_dem_ushouse_2020` that can be explained by `clinton_percent_2016`, the relationship gets way weaker:

In [ ]:
qplot(spending_dem_ushouse_2020, residuals, data = democrats) + 
    geom_smooth(method = 'lm') 

This is basically what multivariate regression does: it gives us the relationship between two variables -- such as spending and Democratic performance in the 2020 elections -- after removing the influence of the other variables in the regression -- such as underlying district partisan lean.

Multivariate regression then does the same thing for the "predictor" variable, too: so the numbers under `Estimate` next to each predictor variable tell us the relationship between _the variation in the outcome variable that cannot be explained by the other predictor variables_ and _the variation in that predictor variable that cannot be explained by the other predictor variables_.

In summary, then, multivariate regression is a way to understand the relationship between two variables while holding other variables constant. This can help us try to reduce omitted variable bias, but it's rare that this can solve omitted variable bias -- we might be missing other omitted variables!